<a href="https://colab.research.google.com/github/kavyanshgangwar/Machine-Translation-Quality-Estimation/blob/main/Btp_neural_net_czech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
FINAL_SCORE = '/content/drive/MyDrive/Colab Notebooks/btp/train.smt.hter'
TRAIN_CZECH_MT = '/content/drive/MyDrive/Colab Notebooks/btp/train.smt.mt'
TRAIN_CZECH_REF = '/content/drive/MyDrive/Colab Notebooks/btp/train.smt.ref'
TRAIN_ENGLISH_SOURCE = '/content/drive/MyDrive/Colab Notebooks/btp/train.smt.src'
TEST_SCORE = '/content/drive/MyDrive/Colab Notebooks/btp/dev.smt.hter'
TEST_CZECH_MT = '/content/drive/MyDrive/Colab Notebooks/btp/dev.smt.mt'
TEST_CZECH_REF = '/content/drive/MyDrive/Colab Notebooks/btp/dev.smt.ref'
TEST_ENGLISH_SOURCE = '/content/drive/MyDrive/Colab Notebooks/btp/dev.smt.src'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
def process(filename):
  with open(filename,'r') as f:
    a = f.readlines()
  for i in range(len(a)):
    a[i] = a[i][0:len(a[i])-1]
  for i in range(len(a)):
    a[i]=a[i].split()
  cnt = 1
  word_to_index_english = {}
  for x in a:
    for y in x:
      if word_to_index_english.get(y) is None:
        word_to_index_english[y]=cnt
        cnt+=1
  sentense_eng = 0
  for x in a:
    sentense_eng = max(sentense_eng,len(x))
  ENGLISH_DATA = np.zeros((len(a),sentense_eng))
  for i in range(len(a)):
    for j in range(len(a[i])):
      ENGLISH_DATA[i][j] = word_to_index_english[a[i][j]]
  return ENGLISH_DATA,word_to_index_english

In [ ]:
def process_test(filename,word_index,train):
  with open(filename,'r') as f:
    a = f.readlines()
  for i in range(len(a)):
    a[i]=a[i][0:len(a[i])-1]
  for i in range(len(a)):
    a[i]=a[i].split()
  DATA = np.zeros((len(a),train.shape[1]))
  unknown = max(word_index.values())+1
  for i in range(len(a)):
    for j in range(min(len(a[i]),train.shape[1])):
      if word_index.get(a[i][j]) is None:
        DATA[i][j] = unknown
      else:
        DATA[i][j] = word_index[a[i][j]]
  return DATA

In [ ]:
def process_czech(filename1,filename2):
  with open(filename1,'r') as f:
    a = f.readlines()
  with open(filename2,'r') as f:
    b = f.readlines()
  for i in range(len(a)):
    a[i] = a[i][0:len(a[i])-1]
  for i in range(len(a)):
    a[i]=a[i].split()
  for i in range(len(b)):
    b[i] =b[i][0:len(b[i])-1]
  for i in range(len(b)):
    b[i]=b[i].split()
  cnt = 1
  word_to_index = {}
  for x in a:
    for y in x:
      if word_to_index.get(y) is None:
        word_to_index[y]=cnt
        cnt+=1
  for x in b:
    for y in x:
      if word_to_index.get(y) is None:
        word_to_index[y]=cnt
        cnt+=1
  sentense_eng = 0
  for x in a:
    sentense_eng = max(sentense_eng,len(x))
  DATA = np.zeros((len(a),sentense_eng))
  for i in range(len(a)):
    for j in range(len(a[i])):
      DATA[i][j] = word_to_index[a[i][j]]
  sentense_1 = 0
  for x in b:
    sentense_1 = max(sentense_1,len(x))
  DATA1 = np.zeros((len(b),sentense_1))
  for i in range(len(b)):
    for j in range(len(b[i])):
      DATA1[i][j] = word_to_index[b[i][j]]
  return DATA,DATA1,word_to_index

In [ ]:
ENGLISH_TRAIN_DATA,word_to_index_english = process(TRAIN_ENGLISH_SOURCE)
CZECH_TRAIN_REF_DATA,CZECH_TRAIN_MT_DATA,word_to_index_czech = process_czech(TRAIN_CZECH_REF,TRAIN_CZECH_MT)

In [ ]:
ENGLISH_TEST_DATA = process_test(TEST_ENGLISH_SOURCE,word_to_index_english,ENGLISH_TRAIN_DATA)
CZECH_TEST_REF_DATA = process_test(TEST_CZECH_REF,word_to_index_czech,CZECH_TRAIN_REF_DATA)
CZECH_TEST_MT_DATA = process_test(TEST_CZECH_MT,word_to_index_czech,CZECH_TRAIN_MT_DATA)
TEST_HTER_SCORE = pd.read_csv(TEST_SCORE,header=None)

In [ ]:
len(word_to_index_czech)

51941

In [ ]:
HTER_SCORE = pd.read_csv(FINAL_SCORE,header=None)

In [ ]:
HTER_SCORE

,0
0,0.454545
1,0.347826
2,0.538462
3,0.500000
4,0.500000
...,...
40249,0.684211
40250,0.347826
40251,0.066667
40252,0.291667


In [ ]:
CZECH_TRAIN_MT_DATA.shape

(40254, 133)

In [ ]:
from tensorflow.keras.layers import Lambda,Input, Embedding, Bidirectional, LSTM, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.metrics import RootMeanSquaredError,MeanAbsoluteError
from keras.callbacks import ModelCheckpoint
import tensorflow_probability as tfp

In [ ]:
filepath = '/content/drive/MyDrive/Colab Notebooks/btp/my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')


In [ ]:
english_input = Input(shape=ENGLISH_TRAIN_DATA.shape[1])
english_embeddings = Embedding(len(word_to_index_english)+2,100)(english_input)
english = Bidirectional(LSTM(128,return_sequences=True))(english_embeddings)
english = LSTM(128)(english)
czech_ref_input = Input(shape=CZECH_TRAIN_REF_DATA.shape[1])
czech_mt_input = Input(shape=CZECH_TRAIN_MT_DATA.shape[1])
czech_input = Concatenate()([czech_mt_input,czech_ref_input])
czech_embeddings = Embedding(len(word_to_index_czech)+2,100)(czech_input)
czech_ref = Lambda(lambda czech_embeddings: czech_embeddings[:,:54,:])(czech_embeddings)
czech_mt = Lambda(lambda czech_embeddings: czech_embeddings[:,54:,:])(czech_embeddings)

czech_ref = Bidirectional(LSTM(128,return_sequences=True))(czech_ref)
czech_ref = LSTM(128)(czech_ref)
czech_mt = Bidirectional(LSTM(128,return_sequences=True))(czech_mt)
czech_mt = LSTM(128)(czech_mt)
x = Concatenate()([czech_ref,czech_mt,english])
x = Dense(64,activation='relu')(x)
x = Dense(32,activation='relu')(x)
x = Dense(8,activation='relu')(x)
x = Dense(1,activation='relu')(x)
model = Model([english_input,czech_ref_input,czech_mt_input],x)

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=[MeanAbsoluteError(),RootMeanSquaredError()])
model.summary()

TypeError: ignored

In [ ]:
r = model.fit(x=[ENGLISH_TRAIN_DATA,CZECH_TRAIN_REF_DATA,CZECH_TRAIN_MT_DATA],y=HTER_SCORE,epochs=20,validation_data=([ENGLISH_TEST_DATA,CZECH_TEST_REF_DATA,CZECH_TEST_MT_DATA],TEST_HTER_SCORE),callbacks=[checkpoint])

In [ ]:
model.save_weights('/content/drive/MyDrive/Colab Notebooks/btp/czech-model')